[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)]
(https://colab.research.google.com/github/KhoaTran235/Vehicle_Speed_Detection/blob/main/demo.ipynb)

In [22]:
!pip install ultralytics

In [23]:
from ultralytics import YOLO

model = YOLO("yolov8n.pt")  # load a pretrained model (recommended for training)
result = model.val(data="coco128.yaml", imgsz=640)  # run validation on COCO dataset at 640x640 resolution

Ultralytics 8.3.249 🚀 Python-3.12.12 torch-2.9.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
YOLOv8n summary (fused): 72 layers, 3,151,904 parameters, 0 gradients, 8.7 GFLOPs
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 1373.6±734.6 MB/s, size: 40.5 KB)
val: Scanning /content/datasets/coco128/labels/train2017.cache... 126 images, 2 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 128/128 278.7Kit/s 0.0s
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 8/8 2.2it/s 3.6s
                   all        128        929      0.639      0.536      0.607      0.448
                person         61        254      0.793      0.677      0.764      0.538
               bicycle          3          6      0.514      0.333      0.315      0.264
                   car         12         46      0.813      0.217      0.272      0.167
            motorcycle          4          5      0.687      0.887      0.898      0.718
              airplane          5  

In [24]:
print(result.box.map)        # mAP50-95
print(result.box.map50)      # mAP50

0.4477848806493725
0.6071667846709387


In [25]:
import torch
import torch.nn.utils.prune as prune

In [26]:
def prune_model(model, amount=0.1):
  for module in model.modules():
    if isinstance(module, torch.nn.Conv2d):
      prune.l1_unstructured(module, name="weight", amount=amount)
      prune.remove(module, "weight")
  return model

In [27]:
torch_model = model.model
print(torch_model)

DetectionModel(
  (model): Sequential(
    (0): Conv(
      (conv): Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
      (act): SiLU(inplace=True)
    )
    (1): Conv(
      (conv): Conv2d(16, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
      (act): SiLU(inplace=True)
    )
    (2): C2f(
      (cv1): Conv(
        (conv): Conv2d(32, 32, kernel_size=(1, 1), stride=(1, 1))
        (act): SiLU(inplace=True)
      )
      (cv2): Conv(
        (conv): Conv2d(48, 32, kernel_size=(1, 1), stride=(1, 1))
        (act): SiLU(inplace=True)
      )
      (m): ModuleList(
        (0): Bottleneck(
          (cv1): Conv(
            (conv): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
            (act): SiLU(inplace=True)
          )
          (cv2): Conv(
            (conv): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
            (act): SiLU(inplace=True)
          )
        )
      )
    )
    (3): Conv(
      (conv): Conv2d(32

In [28]:
pruned_torch_model = prune_model(torch_model, amount=0.1)
print(pruned_torch_model)

DetectionModel(
  (model): Sequential(
    (0): Conv(
      (conv): Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
      (act): SiLU(inplace=True)
    )
    (1): Conv(
      (conv): Conv2d(16, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
      (act): SiLU(inplace=True)
    )
    (2): C2f(
      (cv1): Conv(
        (conv): Conv2d(32, 32, kernel_size=(1, 1), stride=(1, 1))
        (act): SiLU(inplace=True)
      )
      (cv2): Conv(
        (conv): Conv2d(48, 32, kernel_size=(1, 1), stride=(1, 1))
        (act): SiLU(inplace=True)
      )
      (m): ModuleList(
        (0): Bottleneck(
          (cv1): Conv(
            (conv): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
            (act): SiLU(inplace=True)
          )
          (cv2): Conv(
            (conv): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
            (act): SiLU(inplace=True)
          )
        )
      )
    )
    (3): Conv(
      (conv): Conv2d(32

In [29]:
model.model = pruned_torch_model
model.save("pruned_yolov8n.pt")

In [ ]:
model = YOLO("pruned_yolov8n.pt")  # load a pretrained model (recommended for training)
result = model.val(data="coco128.yaml")  # evaluate model performance on the validation set

Ultralytics 8.3.249 🚀 Python-3.12.12 torch-2.9.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 1690.1±918.0 MB/s, size: 44.6 KB)
val: Scanning /content/datasets/coco128/labels/train2017.cache... 126 images, 2 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 128/128 274.5Kit/s 0.0s
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 8/8 1.8it/s 4.4s
                   all        128        929      0.487      0.428      0.464      0.318
                person         61        254      0.335      0.524      0.408      0.244
               bicycle          3          6       0.63      0.333      0.301      0.221
                   car         12         46      0.213      0.152      0.109     0.0478
            motorcycle          4          5      0.523        0.8      0.832      0.651
              airplane          5          6      0.529      0.667      0.683      0.411
                   bus      

In [ ]:
print(result.box.map)        # mAP50-95
print(result.box.map50)      # mAP50